# Extração de Features - 5 Braços Experimentais

Este notebook extrai features para os 5 braços experimentais:

1. **Baseline CNN:** ResNet/U-Net padrão
2. **ViT Puro:** ViT pré-treinado no ImageNet
3. **ViT + Contrastive:** Usando domain_specific_cl
4. **ViT + MIM:** Usando MIM-Med3D
5. **ViT + Sparse:** Aplicação de esparsidade nas features

## Estrutura de Saída

As features serão salvas em:
```
features/
├── baseline_cnn/
├── vit_pure/
├── vit_contrastive/
├── vit_mim/
└── vit_sparse/
```


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from transformers import TFAutoModel, AutoFeatureExtractor

# ============================================
# DETECÇÃO DE AMBIENTE (COLAB OU LOCAL)
# ============================================
try:
    import google.colab
    IN_COLAB = True
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Colab detectado - Drive montado")
except ImportError:
    IN_COLAB = False
    print("✅ Ambiente local detectado")

# Configurar caminhos baseado no ambiente
if IN_COLAB:
    BASE_DIR = Path("/content/drive/MyDrive/Mestrado_TCC")
    DATA_DIR = BASE_DIR / "datasets" / "processed"
    FEATURES_DIR = BASE_DIR / "features"
    # Mudar para diretório do framework
    FRAMEWORK_DIR = BASE_DIR / "Framework"
    if FRAMEWORK_DIR.exists():
        os.chdir(FRAMEWORK_DIR)
else:
    BASE_DIR = Path("../")
    DATA_DIR = BASE_DIR / "data" / "processed"
    FEATURES_DIR = BASE_DIR / "features"

# Configuração de GPU
from tensorflow.keras.mixed_precision import set_global_policy
if len(tf.config.list_physical_devices('GPU')) > 0:
    policy = tf.keras.mixed_precision.Policy('mixed_float16')
    set_global_policy(policy)
    print("✅ GPU ativa - Mixed Precision habilitado")
else:
    print("⚠️  GPU não disponível - Usando CPU")

# Criar diretórios
FEATURES_DIR.mkdir(parents=True, exist_ok=True)

# Criar diretórios para cada braço experimental
EXPERIMENTAL_ARMS = [
    "baseline_cnn",
    "vit_pure",
    "vit_contrastive",
    "vit_mim",
    "vit_sparse"
]

for arm in EXPERIMENTAL_ARMS:
    (FEATURES_DIR / arm).mkdir(exist_ok=True)

# Batch sizes otimizados por ambiente
BATCH_SIZES = {
    "baseline_cnn": 64 if IN_COLAB else 32,
    "vit_pure": 32 if IN_COLAB else 16,
    "vit_contrastive": 16 if IN_COLAB else 8,
    "vit_mim": 8 if IN_COLAB else 4,
    "vit_sparse": 64 if IN_COLAB else 32
}

print(f"\n📁 Diretórios configurados:")
print(f"   BASE_DIR: {BASE_DIR}")
print(f"   FEATURES_DIR: {FEATURES_DIR}")
print(f"\n📊 Batch sizes: {BATCH_SIZES}")


## Funções Auxiliares para Carregamento de Dados Médicos


In [ ]:
import nibabel as nib

def load_medical_image(filepath, normalize=True, slice_idx=None):
    """
    Carrega imagem médica (2D slice de volume 3D ou imagem 2D).
    Suporta formatos .nii.gz, .nii, .jpg, .png
    
    Args:
        filepath: Caminho para o arquivo
        normalize: Se True, normaliza para [0, 255]
        slice_idx: Índice do slice para volumes 3D (None = slice central)
    """
    filepath = Path(filepath)
    
    if filepath.suffix == '.gz' or '.nii' in filepath.name:
        # Arquivo NIfTI
        img = nib.load(str(filepath))
        data = img.get_fdata()
        
        # Se for 3D, pegar slice central ou especificado
        if len(data.shape) == 3:
            if slice_idx is None:
                slice_idx = data.shape[2] // 2
            data = data[:, :, slice_idx]
        
        # Normalizar para [0, 255] e converter para uint8
        if normalize:
            data_min, data_max = data.min(), data.max()
            if data_max > data_min:
                data = (data - data_min) / (data_max - data_min + 1e-8) * 255
            else:
                data = np.zeros_like(data)
        
        # Converter para RGB (repetir canal se necessário)
        if len(data.shape) == 2:
            data = np.stack([data, data, data], axis=-1)
        
        return tf.cast(data, tf.uint8)
    else:
        # Imagem 2D padrão
        image = tf.io.read_file(str(filepath))
        if filepath.suffix.lower() in ['.jpg', '.jpeg']:
            image = tf.image.decode_jpeg(image, channels=3)
        elif filepath.suffix.lower() == '.png':
            image = tf.image.decode_png(image, channels=3)
        return image

def preprocess_image(image, input_size=224, method='bilinear'):
    """Redimensiona e normaliza imagem para o modelo."""
    image = tf.cast(image, tf.float32)
    if len(image.shape) == 2:
        image = tf.expand_dims(image, -1)
        image = tf.repeat(image, 3, axis=-1)
    
    resize_method = tf.image.ResizeMethod.BILINEAR if method == 'bilinear' else tf.image.ResizeMethod.LANCZOS3
    image = tf.image.resize(image, [input_size, input_size], method=resize_method)
    return image

print("Funções auxiliares criadas!")


## Braço 1: Baseline CNN (ResNet50)


In [ ]:
def build_baseline_cnn_extractor(input_size=224):
    """Constrói extrator de features usando ResNet50 pré-treinado no ImageNet."""
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(input_size, input_size, 3)
    )
    base_model.trainable = False
    
    # Global Average Pooling
    inputs = layers.Input(shape=(input_size, input_size, 3), dtype=tf.float32)
    x = tf.keras.applications.resnet50.preprocess_input(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    
    model = models.Model(inputs=inputs, outputs=x)
    return model

print("Extrator Baseline CNN criado!")


## Braço 2: ViT Puro (ImageNet pré-treinado)


In [ ]:
def build_vit_pure_extractor(model_name="google/vit-base-patch16-224", input_size=224):
    """Constrói extrator de features usando ViT pré-treinado no ImageNet."""
    feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
    vit_model = TFAutoModel.from_pretrained(model_name)
    vit_model.trainable = False
    
    def vit_preprocessing(x):
        x = tf.cast(x, tf.float32) / 255.0
        mean = tf.constant(feature_extractor.image_mean, shape=[1, 1, 1, 3], dtype=tf.float32)
        std = tf.constant(feature_extractor.image_std, shape=[1, 1, 1, 3], dtype=tf.float32)
        return (x - mean) / std
    
    inputs = layers.Input(shape=(input_size, input_size, 3), dtype=tf.float32)
    x = vit_preprocessing(inputs)
    
    def call_vit(x):
        outputs = vit_model(pixel_values=x)
        return outputs.last_hidden_state[:, 0]  # token [CLS]
    
    features = layers.Lambda(call_vit, name="vit_features")(x)
    model = models.Model(inputs=inputs, outputs=features)
    return model

print("Extrator ViT Puro criado!")


## Braço 3: ViT + Contrastive (domain_specific_cl)

**Nota:** Este repositório usa TensorFlow 1.x, que é incompatível com versões mais recentes. 
A integração pode requerer um ambiente separado ou adaptação do código.


In [ ]:
def build_vit_contrastive_extractor(repo_path="../repositories/domain_specific_cl", checkpoint_path=None):
    """
    Constrói extrator usando modelo contrastivo do domain_specific_cl.
    
    IMPORTANTE: Este repositório usa TensorFlow 1.x, que requer adaptação especial.
    
    Args:
        repo_path: Caminho para o repositório clonado
        checkpoint_path: Caminho para o checkpoint do modelo pré-treinado
    """
    repo_path = Path(repo_path).absolute()
    
    if not repo_path.exists():
        print("⚠️  Repositório domain_specific_cl não encontrado!")
        print("   Usando ViT puro como fallback")
        return build_vit_pure_extractor()
    
    # Adicionar ao path do Python
    sys.path.insert(0, str(repo_path))
    
    try:
        # Tentar importar módulos do repositório
        # Nota: Isso pode falhar se TensorFlow 1.x não estiver instalado
        import models
        import utils
        
        print("✅ Módulos do domain_specific_cl importados com sucesso")
        print("⚠️  Para usar o modelo contrastivo, você precisa:")
        print("   1. Instalar TensorFlow 1.12.0 em um ambiente separado")
        print("   2. Treinar ou baixar o modelo pré-treinado")
        print("   3. Carregar o checkpoint e extrair features do encoder")
        print("   Por enquanto, usando ViT puro como fallback")
        
        # TODO: Implementar carregamento do modelo quando TensorFlow 1.x estiver disponível
        # Exemplo:
        # cfg = utils.load_config(...)
        # model = models.modelObj(cfg)
        # encoder = model.encoder_network(...)
        # return encoder
        
    except ImportError as e:
        print(f"⚠️  Erro ao importar módulos do domain_specific_cl: {e}")
        print("   Isso é esperado se TensorFlow 1.x não estiver instalado")
        print("   Usando ViT puro como fallback")
    except Exception as e:
        print(f"⚠️  Erro inesperado: {e}")
        print("   Usando ViT puro como fallback")
    
    # Fallback para ViT puro
    return build_vit_pure_extractor()

print("Extrator ViT + Contrastive (com fallback) criado!")


## Braço 4: ViT + MIM (MIM-Med3D)

**Nota:** Este repositório usa PyTorch. Vamos criar um wrapper para converter para TensorFlow ou usar diretamente.


In [ ]:
def build_vit_mim_extractor(repo_path="../repositories/MIM-Med3D", checkpoint_path=None, model_type="ViTSimMIM"):
    """
    Constrói extrator usando modelo MIM do MIM-Med3D.
    
    IMPORTANTE: Este repositório usa PyTorch. A integração pode requerer conversão ou uso direto do PyTorch.
    
    Args:
        repo_path: Caminho para o repositório clonado
        checkpoint_path: Caminho para o checkpoint do modelo pré-treinado
        model_type: Tipo de modelo ("ViTSimMIM", "MAE", "VisionTransformer3D")
    """
    repo_path = Path(repo_path).absolute()
    code_path = repo_path / "code"
    
    if not repo_path.exists():
        print("⚠️  Repositório MIM-Med3D não encontrado!")
        print("   Usando ViT puro como fallback")
        return build_vit_pure_extractor()
    
    # Adicionar ao path do Python
    sys.path.insert(0, str(code_path))
    
    try:
        # Tentar importar módulos do repositório
        import torch
        from models import ViTSimMIM, MAE, VisionTransformer3D
        
        print("✅ Módulos do MIM-Med3D importados com sucesso")
        print("⚠️  Para usar o modelo MIM, você precisa:")
        print("   1. Instalar dependências: pip install -r requirements.txt")
        print("   2. Baixar ou treinar o modelo pré-treinado")
        print("   3. Carregar o checkpoint e extrair features")
        print("   Por enquanto, usando ViT puro como fallback")
        
        # TODO: Implementar carregamento do modelo quando PyTorch estiver disponível
        # Exemplo:
        # if checkpoint_path and os.path.exists(checkpoint_path):
        #     checkpoint = torch.load(checkpoint_path, map_location='cpu')
        #     model = ViTSimMIM(...)
        #     model.load_state_dict(checkpoint['state_dict'])
        #     model.eval()
        #     return model
        # else:
        #     print("Checkpoint não encontrado, usando modelo sem pré-treinamento")
        
    except ImportError as e:
        print(f"⚠️  Erro ao importar módulos do MIM-Med3D: {e}")
        print("   Verifique se PyTorch e dependências estão instaladas")
        print("   Usando ViT puro como fallback")
    except Exception as e:
        print(f"⚠️  Erro inesperado: {e}")
        print("   Usando ViT puro como fallback")
    
    # Fallback para ViT puro
    return build_vit_pure_extractor()

print("Extrator ViT + MIM (com fallback) criado!")


## Braço 5: ViT + Sparse

Aplica esparsidade nas features extraídas pelo ViT puro usando Dictionary Learning.


In [ ]:
from sklearn.decomposition import DictionaryLearning

def apply_sparsity_to_features(features, n_atoms=50, alpha=0.1):
    """
    Aplica esparsidade nas features usando Dictionary Learning.
    
    Args:
        features: Array de features (n_samples, n_features)
        n_atoms: Número de átomos no dicionário
        alpha: Parâmetro de regularização para esparsidade
    """
    # Aprender dicionário
    dict_learner = DictionaryLearning(
        n_components=n_atoms,
        alpha=alpha,
        fit_algorithm='lars',
        transform_algorithm='lasso_lars',
        n_jobs=-1
    )
    
    # Aprender dicionário e transformar features
    sparse_features = dict_learner.fit_transform(features)
    
    return sparse_features, dict_learner

print("Função de esparsidade criada!")


## Função de Extração Genérica

Esta função extrai features de um dataset usando qualquer um dos modelos acima.


In [ ]:
def extract_features_from_files(model, file_list, labels_list, output_path, batch_size=32, input_size=224):
    """
    Extrai features de uma lista de arquivos usando um modelo.
    
    Args:
        model: Modelo de extração de features (TensorFlow/Keras)
        file_list: Lista de caminhos para arquivos de imagem
        labels_list: Lista de labels correspondentes
        output_path: Caminho para salvar as features (.npy)
        batch_size: Tamanho do batch
        input_size: Tamanho da imagem de entrada
    """
    features_list = []
    labels_array = []
    
    print(f"Extraindo features de {len(file_list)} arquivos...")
    
    # Processar em batches
    for i in range(0, len(file_list), batch_size):
        batch_files = file_list[i:i+batch_size]
        batch_labels = labels_list[i:i+batch_size]
        
        # Carregar e pré-processar imagens
        batch_images = []
        for filepath in batch_files:
            try:
                image = load_medical_image(filepath, normalize=True)
                image = preprocess_image(image, input_size=input_size)
                batch_images.append(image)
            except Exception as e:
                print(f"Erro ao carregar {filepath}: {e}")
                continue
        
        if len(batch_images) == 0:
            continue
        
        # Converter para tensor
        batch_images = tf.stack(batch_images)
        
        # Extrair features
        batch_features = model.predict(batch_images, verbose=0)
        features_list.append(batch_features)
        labels_array.extend(batch_labels[:len(batch_features)])
        
        if (i + batch_size) % (batch_size * 10) == 0:
            print(f"  Processados {min(i + batch_size, len(file_list))}/{len(file_list)} arquivos...")
    
    # Concatenar todas as features
    if len(features_list) > 0:
        all_features = np.concatenate(features_list, axis=0)
        all_labels = np.array(labels_array)
        
        # Salvar
        output_path = Path(output_path)
        output_path.parent.mkdir(parents=True, exist_ok=True)
        np.save(str(output_path), all_features)
        np.save(str(output_path).replace('_features.npy', '_labels.npy'), all_labels)
        
        print(f"✅ Features salvas em: {output_path}")
        print(f"   Shape: {all_features.shape}")
        return all_features, all_labels
    else:
        print("⚠️  Nenhuma feature foi extraída!")
        return None, None

print("Função de extração genérica criada!")


## Exemplo de Uso

Aqui está um exemplo de como usar as funções acima para extrair features de um dataset.


In [ ]:
# Exemplo: Extrair features com ViT Puro
# Descomente e ajuste os caminhos conforme necessário

# # 1. Construir modelo
# vit_model = build_vit_pure_extractor()
# 
# # 2. Preparar lista de arquivos (exemplo)
# # file_list = list((DATA_DIR / "ACDC" / "training").glob("**/*.nii.gz"))
# # labels_list = [0] * len(file_list)  # Ajustar conforme necessário
# 
# # 3. Extrair features
# # features, labels = extract_features_from_files(
# #     model=vit_model,
# #     file_list=file_list,
# #     labels_list=labels_list,
# #     output_path=FEATURES_DIR / "vit_pure" / "train_features.npy",
# #     batch_size=32,
# #     input_size=224
# # )

print("Exemplo de uso preparado. Descomente e ajuste conforme necessário.")
